<a href="https://colab.research.google.com/github/maimuna-noshin/Solar_Prediction/blob/main/solarprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [115]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, BatchNormalization, Flatten
from sklearn.preprocessing import StandardScaler
from scipy.fftpack import fft
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [116]:
df=pd.read_csv("/content/updated_dataset.csv")

In [117]:
# Convert to datetime
df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])


# Get the index of 'hour' in X_test
hour_column_index = list(df.columns).index("Hour")

# Apply nighttime condition to predicted values
nighttime_indices = (X_test[:, hour_column_index] < 6) | (X_test[:, hour_column_index] >= 18)
y_pred[nighttime_indices, :] = 0  # Set all nighttime predictions to 0


In [118]:
# Selecting features (X) and target variables (Y)
features = ['Hour', 'Temperature_A','Pressure_A', 'Day','Month', 'Surface Albedo_A', 'Cloud Type_A']
targets = ['DNI_A']  # Target variables for all three locations

In [119]:

X = df[features].values
y = df[targets].values

In [120]:
# Keep timestamps before scaling
timestamps = df[['Month', 'Day', 'Hour']]

In [121]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler(feature_range=(0, 1000))
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
# Save the feature scaler
with open("scaler_X.pkl", "wb") as f:
    pickle.dump(scaler_X, f)

# Save the target scaler
with open("scaler_y.pkl", "wb") as f:
    pickle.dump(scaler_y, f)

In [122]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test, timestamps_train, timestamps_test = train_test_split(X_scaled, y_scaled,timestamps, test_size=0.2, random_state=42)
print(X_train.max())

1.0


In [123]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(targets), activation='linear')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [124]:
# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [125]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

Epoch 1/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 70052.1719 - mae: 198.1021 - val_loss: 36843.4648 - val_mae: 146.7770
Epoch 2/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 37029.5664 - mae: 132.4153 - val_loss: 30259.0430 - val_mae: 109.3130
Epoch 3/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 31495.7246 - mae: 110.6573 - val_loss: 28803.2539 - val_mae: 98.2632
Epoch 4/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 30303.7578 - mae: 105.4251 - val_loss: 28267.1523 - val_mae: 97.9566
Epoch 5/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 30539.1953 - mae: 105.5550 - val_loss: 27882.9336 - val_mae: 95.9817
Epoch 6/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 28028.6094 - mae: 98.0760 - val_loss: 27664.1035 - val_mae: 97.5003
Epoch 7/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 28906.4531 - mae: 100.1041 - val_loss: 27590.2812 - val_mae: 97.8399
Epoch 8/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 29617.9082 - mae: 102.9937 - val_los

In [126]:
# Predict future solar irradiance for all three locations
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
# Set negative irradiance values to 0
y_pred[y_pred < 10] = 0
# Apply nighttime condition to predicted values
nighttime_indices = X_test[:, 1] == 0  # Check if the 'hour_sin' feature indicates nighttime
y_pred[nighttime_indices, :] = 0  # Set all predictions to 0 during nighttime

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [127]:
# Convert predictions into a DataFrame
predictions_df = timestamps_test.copy()

# Assign each column separately
predictions_df[['DNI_A']] = y_pred

In [128]:
# Print first 100 predictions with date and hour
print(predictions_df.head(100))
predictions_df.to_csv("solar_irradiance_predictions.csv", index=False)  # Saves to a CSV]

      Month  Day  Hour       DNI_A
6056      9   10     8  291.546112
5556      8   20    12  291.910614
5990      9    7    14  304.818207
7674     11   16    18   14.384103
3319      5   19     7  184.611221
...     ...  ...   ...         ...
6006      9    8     6   60.272686
8063     12    2    23    0.000000
14        1    1    14  449.599213
5306      8   10     2    0.000000
8524     12   22     4    0.000000

[100 rows x 4 columns]


In [129]:
# Save using the new format:
model.save("solar_irradiance_predictions.keras")  # Or model.save("my_model") - the .keras is optional

# Load using the new format:
model = keras.models.load_model("solar_irradiance_predictions.keras")  # Or model = keras.models.load_model("my_model")

In [130]:
#  Load the scalers used during training

with open("scaler_X.pkl", "rb") as f:
    scaler_X = pickle.load(f)
with open("scaler_y.pkl", "rb") as f:
    scaler_y = pickle.load(f)

In [131]:
# Generate timestamps for the first week of March 2025 (hourly)
date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")
future_df = pd.DataFrame({"datetime": date_range})

<ipython-input-131-0e223ba96d09>:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")


In [132]:
#  Feature Engineering
future_df["day"] = future_df["datetime"].dt.day
future_df["month"] = future_df["datetime"].dt.month
future_df["hour"] = future_df["datetime"].dt.hour
future_df.drop(columns=["datetime"], inplace=True)

In [133]:
# Generate random values
future_df["Temperature_A"] = np.random.uniform(27, 30, size=168)  # Temperature between 15°C to 30°C
future_df["Pressure_A"] = np.random.uniform(1000, 1050, size=168)  # Pressure between 950-1050 hPa
future_df["Surface Albedo_A"] = np.random.uniform(0.1, 0.5, size=168)  # Albedo between 0.1 - 0.5
future_df["Cloud Type_A"] = np.random.randint(0, 4, size=168)  # Cloud types (0-3)

# Merge the first week’s random values with `future_df`
#future_df = future_df.merge(future_df, on=["Month", "Day"], how="left")

# Time-based encoding (sin/cos transformation)
#future_df["hour_sin"] = np.sin(2 * np.pi * future_df["hour"] / 24)
#future_df["hour_cos"] = np.cos(2 * np.pi * future_df["hour"] / 24)

# Select the same features used in training
#X_future = future_df[['hour_sin', 'hour_cos', 'Temperature_A', 'Pressure_A', 'day', 'Month', 'Surface Albedo_A', 'Cloud Type_A']].values

In [134]:
#  Select the same features used in training
X_future = future_df[['hour', 'Temperature_A','Pressure_A', 'day','month', 'Surface Albedo_A', 'Cloud Type_A']].values
#  Scale future data using the trained scaler
X_future_scaled = scaler_X.transform(X_future)
print(X_future_scaled)


[[0.         0.58787757 1.68448475 ... 0.18181818 6.05430527 0.33333333]
 [0.04347826 0.61163534 0.96639644 ... 0.18181818 1.78556954 0.11111111]
 [0.08695652 0.61309406 1.14198487 ... 0.18181818 2.63360042 0.        ]
 ...
 [0.91304348 0.60043754 1.72317233 ... 0.18181818 5.77649241 0.11111111]
 [0.95652174 0.60611947 1.02223404 ... 0.18181818 3.47656151 0.33333333]
 [1.         0.58899184 1.10192443 ... 0.18181818 2.37173096 0.22222222]]


In [135]:
# Get the index of 'hour' in X_test
hour_column_index = list(future_df.columns).index("hour")

# Apply nighttime condition to predicted values
nighttime_indices = (X_test[:, hour_column_index] < 6) | (X_test[:, hour_column_index] >= 18)
y_pred[nighttime_indices, :] = 0  # Set all nighttime predictions to 0


In [136]:
#  Make predictions
y_future_scaled = model.predict(X_future_scaled)

#  Inverse scale predictions
y_future = scaler_y.inverse_transform(y_future_scaled)
#  Set negative irradiance values to 0
y_future[y_future < 10] = 0

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


In [137]:
# Save predictions to DataFrame
future_df[['DNI_A']] = y_future

#  Save to CSV
future_df.to_csv("Future_Solar_Predictions_March2025.csv", index=False)

#Display first 10 predictions
print(future_df.head(10))

   day  month  hour  Temperature_A   Pressure_A  Surface Albedo_A  \
0    1      3     0      27.364809  1041.534542          0.442715   
1    1      3     1      28.139312  1019.991893          0.229278   
2    1      3     2      28.186866  1025.259546          0.271680   
3    1      3     3      28.005739  1013.686359          0.118502   
4    1      3     4      29.262112  1027.088455          0.121155   
5    1      3     5      28.614389  1037.570942          0.420380   
6    1      3     6      28.376987  1044.456238          0.320006   
7    1      3     7      27.683032  1012.599798          0.288832   
8    1      3     8      28.544478  1045.403997          0.445132   
9    1      3     9      28.724305  1000.329656          0.211176   

   Cloud Type_A       DNI_A  
0             3   15.985423  
1             1    0.000000  
2             0   21.353054  
3             2    0.000000  
4             1   75.469765  
5             1   50.490204  
6             0  229.785202  
